In [2]:
from dotenv import load_dotenv, dotenv_values
import os

# Load environment variables from .env file
config = dotenv_values("C:/Users/SACHENDRA/Documents/Activeloop/.env")
load_dotenv("C:/Users/SACHENDRA/Documents/Activeloop/.env")

True

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.graphs.networkx_graph import KG_TRIPLE_DELIMITER

# Prompt template for knowledge triple extraction
_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE = (
    "You are a networked intelligence helping a human track knowledge triples"
    " about all relevant people, things, concepts, etc. and integrating"
    " them with your knowledge stored within your weights"
    " as well as that stored in a knowledge graph."
    " Extract all of the knowledge triples from the text."
    " A knowledge triple is a clause that contains a subject, a predicate,"
    " and an object. The subject is the entity being described,"
    " the predicate is the property of the subject that is being"
    " described, and the object is the value of the property.\n\n"
    "EXAMPLE\n"
    "It's a state in the US. It's also the number 1 producer of gold in the US.\n\n"
    f"Output: (Nevada, is a, state){KG_TRIPLE_DELIMITER}(Nevada, is in, US)"
    f"{KG_TRIPLE_DELIMITER}(Nevada, is the number 1 producer of, gold)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "I'm going to the store.\n\n"
    "Output: NONE\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "Oh huh. I know Descartes likes to drive antique scooters and play the mandolin.\n"
    f"Output: (Descartes, likes to drive, antique scooters){KG_TRIPLE_DELIMITER}(Descartes, plays, mandolin)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "{text}"
    "Output:"
)

KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT = PromptTemplate(
    input_variables=["text"],
    template=_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE,
)

# Make sure to save your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
# Instantiate the OpenAI model
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.9)

# Create an LLMChain using the knowledge triple extraction prompt
chain = LLMChain(llm=llm, prompt=KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT)

# Run the chain with the specified text
text = "The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris."
triples = chain.run(text)

print(triples)

C:\Users\SACHENDRA\miniconda3\envs\activeloop2\Lib\site-packages\langchain\llms\openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\SACHENDRA\miniconda3\envs\activeloop2\Lib\site-packages\langchain\llms\openai.py:677: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


(Paris, is the capital of, France)<|>(Paris, is the most populous city of, France)<|>(Eiffel Tower, is a famous landmark in, Paris)


In [4]:
# Collecting triplets as a list
def parse_triples(response, delimiter=KG_TRIPLE_DELIMITER):
    if not response:
        return []
    return response.split(delimiter)

triples_list = parse_triples(triples)

# Print the extracted relation triplets
print(triples_list)

['(Paris, is the capital of, France)', '(Paris, is the most populous city of, France)', '(Eiffel Tower, is a famous landmark in, Paris)']


In [5]:
!pip install pyvis

   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   - ------------------------------------- 20.5/756.0 kB 330.3 kB/s eta 0:00:03
   ---- ---------------------------------- 81.9/756.0 kB 919.0 kB/s eta 0:00:01
   ------- -------------------------------- 143.4/756.0 kB 1.1 MB/s eta 0:00:01
   ---------- ----------------------------- 194.6/756.0 kB 1.2 MB/s eta 0:00:01
   ----------- ---------------------------- 215.0/756.0 kB 1.2 MB/s eta 0:00:01
   ----------- ---------------------------- 215.0/756.0 kB 1.2 MB/s eta 0:00:01
   -------------------- ------------------- 378.9/756.0 kB 1.2 MB/s eta 0:00:01
   ----------------------- ---------------- 440.3/756.0 kB 1.3 MB/s eta 0:00:01
   -------------------------- ------------- 491.5/756.0 kB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 522.2/756.0 kB 1.3 MB/s eta 0:00:01
   --------------------------- ------------ 522.2/756.0 kB 1.3 MB/s eta 0:00:01
   ------------------------------------ --- 686.1

In [20]:
'''
Knowledge Graph Visualizations using pyvis library
'''
from pyvis.network import Network
import networkx as nx

# Create a NetworkX graph from the extracted relation triplets
def create_graph_from_triplets(triplets):
    G = nx.DiGraph()
    for triplet in triplets:
        subject, predicate, obj = triplet.strip().split(',')
        G.add_edge(subject.strip(), obj.strip(), label=predicate.strip())
        print("Edges in Graph:", G.edges(data=True))
    return G
    # for triplet in triplets:
    #     # Remove leading and trailing spaces, and strip parentheses
    #     subject, predicate, obj = [part.strip().strip('()') for part in triplet.split(',')]
    #     print(subject, predicate, obj)

    #     # Ensure proper handling of spaces in the predicate
    #     predicate = predicate.replace("_", " ")  # Replacing underscores with spaces
    #     print("Processed Predicate:", predicate)

    #     # Add the edge to the graph
    #     G.add_edge(subject, obj, label=predicate)
        
    #     print("Edges in Graph:", G.edges(data=True))
    return G

# Convert the NetworkX graph to a PyVis network
def nx_to_pyvis(networkx_graph):
    pyvis_graph = Network(notebook=True)
    for node in networkx_graph.nodes():
        pyvis_graph.add_node(node)
    for edge in networkx_graph.edges(data=True):
        pyvis_graph.add_edge(edge[0], edge[1], label=edge[2]["label"])
    return pyvis_graph

triplets = [t.strip() for t in triples_list if t.strip()]
print(triplets)
graph = create_graph_from_triplets(triplets)
pyvis_network = nx_to_pyvis(graph)

# Customize the appearance of the graph
pyvis_network.toggle_hide_edges_on_drag(True)
pyvis_network.toggle_physics(False)
pyvis_network.set_edge_smooth('discrete')

# Show the interactive knowledge graph visualization
pyvis_network.show('knowledge_graph.html')

['(Paris, is the capital of, France)', '(Paris, is the most populous city of, France)', '(Eiffel Tower, is a famous landmark in, Paris)']
Edges in Graph: [('(Paris', 'France)', {'label': 'is the capital of'})]
Edges in Graph: [('(Paris', 'France)', {'label': 'is the most populous city of'})]
Edges in Graph: [('(Paris', 'France)', {'label': 'is the most populous city of'}), ('(Eiffel Tower', 'Paris)', {'label': 'is a famous landmark in'})]
knowledge_graph.html
